<p align="center">
    <a href="https://bit.ly/3fsA9RL"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>
</p>

<div class="alert alert-info">

If you encounter issues you cannot resolve, simply ask in our [Slack community](https://join.slack.com/t/lineacommunity/shared_invite/zt-18kizfn3b-1Qu_HDT3ahGudnAwoFAw9Q)'s `#support` channel. We are always happy and ready to help you!

</div>

Let's install and load LineaPy, along with other packages.

In [ ]:

%%capture
!pip -q install lineapy~=0.2 scikit-learn pandas matplotlib

In [1]:

%load_ext lineapy

In [ ]:
import lineapy
lineapy.tag("Discover past work demo")

# Use LineaPy to Discover and Trace Past Work

## Scenario

Data science is often a team effort where one person's work uses results from another. For instance, as a data scientist building a model, we may use various features pre-processed by other colleagues.

## What might happen next?

In using results generated by other people, we may encounter issues such as missing values, numbers that look suspicious, and uninformative/unintelligible variable names. If so, we may want/need to check how these results came into being in the first place. Often, this means tracing back the code that was used to generate the result in question (e.g., feature table). In practice, this can become a challenging task because

* It may not be clear who produced the result.

* Even if we know who to ask, the person may not remember where the exact version of the code is.

* The person may have overwritten the code without version control.

* The person may no longer be in the organization with no proper handover of the relevant knowledge.

This then makes it almost impossible to identify the root of the issue, which may render the result unreliable and even unusable.

## How can LineaPy help here?

In LineaPy, any intermediate result from a data science process (e.g., tables, models) can be stored as an "artifact", which encapsulates both value and code (you can read more about LineaPy artifacts [here](https://docs.lineapy.org/en/main/concepts/artifact.html#artifact)). Hence, it becomes very easy to trace back what code produced the result in question, like so:

```python
import lineapy

# List all artifacts
lineapy.artifact_store()

# Retrieve the desired artifact
artifact = lineapy.get("artifact_name")

# Retrieve code that generated the artifact
print(artifact.get_code())
```

## What will we learn in rest of the notebook?

In the demo below, we will walk through a hands-on example of discovering and tracing past code using LineaPy artifacts. Specifically, we start with an issue with a pre-trained model we want to use, and then systematically trace back the past code to identify the root cause of the issue.

We strongly encourage you to try this demo on your own and check out the official [documentation](https://docs.lineapy.org/en/main/index.html) to learn more use cases of LineaPy.

## Demo

### Accessing Shared Artifact Store

With proper setup, artifacts can be shared across multiple users in different locations. For instance, once we provision PostgreSQL database and S3 bucket, we can configure and use a shared artifact store on AWS, like so:

```python
# Use Amazon RDS as database for artifact metadata
PASSWORD = getpass()
lineapy.options.set("database_url", "[POSTGRES-DB-CONNECTION-STRING]")

# Store artifacts in Amazon S3
lineapy.options.set("artifact_storage_dir", "[S3-BUCKET-URI]")
```

where `"[POSTGRES-DB-CONNECTION-STRING]"` and `"[S3-BUCKET-URI]"` are to be replaced with appropriate actual values.

For access and reproducibility, however, this demo will instead use a “frozen” copy of a pre-populated artifact store in our public repo. For a live demo of artifact sharing on cloud, check our workshop recording here: [LINK-WHEN-AVAILABLE].

In [2]:
# Download artifact from repo

import pathlib
import shutil
import zipfile
import requests

# Clean up existing folder
if pathlib.Path('demos').exists():
    shutil.rmtree('demos')
if pathlib.Path('/tmp/.lineapy').exists():
    shutil.rmtree('/tmp/.lineapy')

# Download artifact store file and extract to local path
url = "https://raw.githubusercontent.com/LineaLabs/lineapy/v0.2.x/examples/use_cases/discover_and_trace_past_work/artifact_store.zip"
with requests.get(url, stream=True) as r:
    r.raise_for_status()
    with open('artifact_store.zip', 'wb') as f:
        for chunk in r.iter_content(chunk_size=16384): 
            f.write(chunk)

with zipfile.ZipFile('artifact_store.zip', 'r') as zip_ref:
    zip_ref.extractall('/tmp')

In [3]:

# Use copied artifact store
lineapy.options.set("database_url", "sqlite:////tmp/.lineapy/db.sqlite")
lineapy.options.set("artifact_storage_dir", "/tmp/.lineapy/linea_pickles")

LineaPy database is changed to sqlite:////tmp/.lineapy/db.sqlite, resetting     
notebook session in next cell                                                   


### Spotting model misbehavior

Now, consider the case where we are trying to use a pre-trained model to make predictions. We can list past artifacts to see which models are available.

In [1]:

# List past artifacts
lineapy.artifact_store()

iris_preprocessed:0 created on 2022-09-29 01:22:39.612871
iris_preprocessed:1 created on 2022-09-29 01:22:41.336159
iris_preprocessed:2 created on 2022-09-29 01:22:43.511112
iris_model:0 created on 2022-09-29 01:22:45.381132
iris_model:1 created on 2022-09-29 01:22:46.786414
iris_model:2 created on 2022-09-29 01:22:47.990517
iris_model:3 created on 2022-09-29 01:22:49.366484
toy_artifact:0 created on 2022-09-29 01:22:50.189060
toy_artifact:1 created on 2022-09-29 01:22:50.676276
toy_artifact:2 created on 2022-09-29 01:22:51.084704

Say we decide to use the latest version of "iris_model", so we retrieve it.

In [2]:

# Retrieve desired model artifact
model_artifact = lineapy.get("iris_model", version=2)
model = model_artifact.get_value()

We can then use the model to make predictions. Specifically, we are trying to use an iris's petal width to predict its sepal width.

In [3]:
import pandas as pd

# Enter data to make predictions on
df = pd.DataFrame({
    "petal.width": [1.3, 5.2, 0.3, 1.5, 4.9],
    "d_versicolor": [1, 0, 0, 1, 0],
    "d_virginica": [0, 1, 0, 0, 1],
})

# Check
df

,petal.width,d_versicolor,d_virginica
0,1.3,1,0
1,5.2,0,1
2,0.3,0,0
3,1.5,1,0
4,4.9,0,1


In [4]:
# Predict
df["sepal.width.pred"] = model.predict(df)

# Check
df

,petal.width,d_versicolor,d_virginica,sepal.width.pred
0,1.3,1,0,1.3
1,5.2,0,1,5.2
2,0.3,0,0,0.3
3,1.5,1,0,1.5
4,4.9,0,1,4.9


Now, we notice there is something suspicious going on here. Why do we get the exact same value for the predictor and its prediction? It appears that the model had been trained incorrectly. Can we get a better insight into how the model got trained?

### Tracing past code

With a LineaPy artifact, this becomes as easy as the following:

In [5]:
# Get code that generated the model artifact
print(model_artifact.get_code())

import lineapy
from sklearn.linear_model import LinearRegression

art_df_processed = lineapy.get("iris_preprocessed", version=2)
df_processed = art_df_processed.get_value()
mod = LinearRegression()
mod.fit(
    X=df_processed[["petal.width", "d_versicolor", "d_virginica"]],
    y=df_processed["sepal.width"],
)



Based on the code above, there does not seem to be an issue in the training process.

The culprit then moves to the data itself, which the code shows had been imported as an artifact itself (during model training). Let's retrieve the corresponding artifact.

In [6]:

# Retrieve desired data artifact
data_artifact = lineapy.get("iris_preprocessed", version=2)
data = data_artifact.get_value()

In [7]:
# Check
data

,sepal.length,sepal.width,petal.length,petal.width,variety,variety_color,d_versicolor,d_virginica
0,5.1,0.2,1.4,0.2,Setosa,green,0,0
1,4.9,0.2,1.4,0.2,Setosa,green,0,0
2,4.7,0.2,1.3,0.2,Setosa,green,0,0
3,4.6,0.2,1.5,0.2,Setosa,green,0,0
4,5.0,0.2,1.4,0.2,Setosa,green,0,0
...,...,...,...,...,...,...,...,...
145,6.7,2.3,5.2,2.3,Virginica,red,0,1
146,6.3,1.9,5.0,1.9,Virginica,red,0,1
147,6.5,2.0,5.2,2.0,Virginica,red,0,1
148,6.2,2.3,5.4,2.3,Virginica,red,0,1


Upon inspection, we notice that `sepal.width` and `petal.width` columns have exactly the same values, which certainly looks like a fluke! How did this come about? Is this an inherent error in the source data? Let's dig the code for this data artifact.

In [8]:
# Get code that generated the data artifact
print(data_artifact.get_code())

import pandas as pd

df = pd.read_csv(
    "https://raw.githubusercontent.com/LineaLabs/lineapy/main/examples/tutorials/data/iris.csv"
)
color_map = {"Setosa": "green", "Versicolor": "blue", "Virginica": "red"}
df["variety_color"] = df["variety"].map(color_map)
df["sepal.width"] = df["petal.width"]
df["d_versicolor"] = df["variety"].apply(lambda x: 1 if x == "Versicolor" else 0)
df["d_virginica"] = df["variety"].apply(lambda x: 1 if x == "Virginica" else 0)



Finally, we are able to identify the root of the issue: for some reason, the data pre-processing code is "corrupted" with an erroneous operation:

```python
df["sepal.width"] = df["petal.width"]
```

But why was this inserted? We can check the entire session code for the data artifact, which includes any comments by the author:

In [10]:

# Get full session code that generates the data artifact
print(data_artifact.get_session_code())

# List past artifacts
lineapy.artifact_store()
import os
import lineapy
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

# Load data
df = pd.read_csv("https://raw.githubusercontent.com/LineaLabs/lineapy/main/examples/tutorials/data/iris.csv")

# View data
df

# Plot petal vs. sepal width
df.plot.scatter("petal.width", "sepal.width")
plt.show()

# Calculate correlation coefficient
df[["petal.width", "sepal.width"]].corr(method="pearson")

# Map each species to a color
color_map = {"Setosa": "green", "Versicolor": "blue", "Virginica": "red"}
df["variety_color"] = df["variety"].map(color_map)

# Plot petal vs. sepal width by species
df.plot.scatter("petal.width", "sepal.width", c="variety_color")
plt.show()

# Check species and their counts
df["variety"].value_counts()

# Create dummy variables encoding species
df["d_versicolor"] = df["variety"].apply(lambda x: 1 if x == "Versicolor" else 0)
df["d_virginica"] = df["variety"].apply(lamb

With some digging, we spot the line:

```python
# Swap variable
df["sepal.width"] = df["petal.width"]
```

Based on the comment, it appears that the author was trying to swap values between `sepal.width` and `petal.width` columns, but missed some steps in doing so. We may not fully understand why this operation was necessary, but we now better understand what caused the model misbehavior. More importantly, we can now fix the error with confidence.

## Recap

In this demo, we saw that LineaPy can make it easy to discover and systematically inspect past work. This is important for increasing institutional knowledge, which in turn can boost the organization's insight and efficiency over time.

To learn more about LineaPy, check out the project [documentation](https://docs.lineapy.org/en/main/index.html).